In [37]:
import csv
import numpy as np
from datetime import datetime

# Constants

In [16]:
STATION_ID_COL_NAME = 'station id'
STATION_NAME_COL_NAME = 'station name'
PARAM_T = 'T [°C]'
PARAM_RAIN = 'količina padavin [mm]'

# Initial Configuration

# Utils

In [17]:
def ARSO_datetime_to_epoch(datetime_string):
    """Convert ARSO datetime into epoch time in seconds"""
    utc_time = datetime.strptime(datetime_string, "%Y-%m-%d %H:%M")
    epoch_time = (utc_time - datetime(1970, 1, 1)).total_seconds()
    return epoch_time

def ARSO_datetime_to_epoch_array(datetime_strings):
    """Convert list of ARSO datetimes into epoch time in seconds"""
    return [ARSO_datetime_to_epoch(dt) for dt in datetime_strings]

# Library

In [54]:
class Station:
    def __init__(self, station_id, station_name, header, data, data_timestamp_epoch):
        self.station_is = station_id
        self.station_name = station_name
        self.header = header
        self.data = data
        self.size = len(data)
        self.data_ts = data_timestamp_epoch
    
    def __str__(self):
        return self.station_id + " " + self.station_name + " size=" + str(self.size)
    
    def __repr(self):
        return self.station_id + " " + self.station_name + " size=" + str(self.size)

In [83]:
def test_load_ksedlo():
    """Load weather data for Korenjsko sedlo that has been dowloaded 
    from ARSO website."""
    print('Start loading korenjsko sedlo')
    import csv
    test_file_ksedlo = '/Users/ales/Desktop/STUDIJ/DodatnoLeto/MatematikaZracunalnikom/WeatherDataTest/KorenjskoSedloTest'
    # open file and read content in a csv format
    csvfile = open(test_file_ksedlo, 'r')
    reader = csv.reader(csvfile, delimiter=',')
    data = list(reader)
    # extract header and clean up data by removing empty lines
    header_line = [el.strip() for el in data[0]]
    station_id = header_line[0]
    station_name = header_line[1]
    header = header_line[2:]
    # there are empty lines generated by parsing that needs to be removed
    data_full = data[2:len(data):2]
    data_empty = data[1:len(data):2]
    # check if 'empty' lines are really empty
    for lst in data_empty:
        if len(lst) != 0:
            # there is a non empty line
            print("Warning: not empty line " + lst)
            return None
    # remove station_id column, station_name column and datatime 
    # and create numpy matrix with floats
    data_clean = []
    for line in data_full:
        try:
            line_clean_float = list(map(float, line[3:]))
            data_clean.append(line_clean_float)
        except:
            print(line[3:])
    data_clean_np = np.array(data_clean)
    # extract datetime and convert it into timestamp epoch
    data_timestamps = np.array([[ARSO_datetime_to_epoch(line[2])] 
                                for line in data_full])
    # finish
    print('Loading successful')
    return Station(station_id, station_name, header, data_clean_np, data_timestamps)

In [84]:
def select_params_from_station(station, params**):
    [stationstation.header.index(param) for param in params]

SyntaxError: invalid syntax (<ipython-input-84-4afe997cc958>, line 1)

# Load Data

In [85]:
# Load training data ksedlo and power prices
ksedlo_station = test_load_ksedlo()
#consumption = DataLoader.test_load_borzen_consumption()
#Y = DataLoader.test_load_borzen_price()

Start loading korenjsko sedlo
['0.1', '']
['0.2', '']
['0.3', '']
['0.5', '']
['0.5', '']
['0.4', '']
['0', '']
['0', '']
['-0.1', '']
['-0.2', '']
['-0.5', '']
['-0.6', '']
['-0.6', '']
['-0.6', '']
['-0.6', '']
['-0.6', '']
['-0.6', '']
['-0.7', '']
['-0.8', '']
['-0.8', '']
['-0.8', '']
['-0.8', '']
['-0.9', '']
['-0.9', '']
['-1.2', '']
['-1.2', '']
['-1.3', '']
['-1.4', '']
['-1.4', '']
['-1.5', '']
['-1.5', '']
['-1.5', '']
['-1.4', '']
['-1.5', '']
['-1.4', '']
['-1.2', '']
['-1', '']
['-0.9', '']
['-0.8', '']
['-0.6', '']
['-0.5', '']
['-0.4', '']
['-0.1', '']
['0.4', '']
['0.9', '']
['1.8', '']
['1.7', '']
['1.5', '']
['1.6', '']
['1.9', '']
['0.9', '']
['1.8', '']
['2.8', '']
['2.5', '']
['2.5', '']
['2.4', '']
['2.7', '']
['3.2', '']
['2.8', '']
['2.9', '']
['2.8', '']
['2.4', '']
['2.1', '']
['1.4', '']
['0.5', '']
['0.4', '']
['-0.1', '']
['-0.5', '']
['-0.6', '']
['-0.5', '']
['-1', '']
['-1.3', '']
['-1.3', '']
['-1.3', '']
['-2.2', '']
['-2.3', '']
['-1.4', '']
['-2.1',

In [87]:
# Check data and remove unnecessary columns
ksedlo_station.data
ksedlo_station.size

1440

In [36]:
ksedlo.header

['station id', 'station name', 'valid', 'T [°C]', 'količina padavin [mm]']